In [45]:
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama import OllamaEmbeddings, ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever
from tqdm.notebook import tqdm
import time
import itertools
from ragas import EvaluationDataset, evaluate
from ragas.run_config import RunConfig
from ragas.metrics import faithfulness, answer_relevancy, answer_correctness, context_precision, context_recall, answer_similarity
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.llms import LangchainLLMWrapper
from datasets import Dataset
import json

In [2]:
local_path = "../pdf/Mateo Wheeler Resume v2.pdf"

if local_path:
    loader = UnstructuredPDFLoader(file_path=local_path)
    data = loader.load()
else:
    print("Upload a PDF file for processing.")

In [3]:
len(data[0].page_content)

6828

In [4]:
#Split and chunk the data
chunk_size = 150
chunk_overlap = 75

text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
chunks = text_splitter.split_documents(data)

# Add the chunks to vector database, which uses nomic for model embeddings
vector_db = Chroma.from_documents(
                                    documents=chunks, 
                                    embedding=OllamaEmbeddings(model="nomic-embed-text"),
                                    collection_name="local-rag"
                                )

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [5]:
local_llm = "llama3.2"
llm = ChatOllama(model=local_llm)

# Set up a basic PromptTemplate as the backbones of the solution
QUERY_PROMPT = PromptTemplate(
    input_variables = ["question"],
        template="""As a recruiter, you need to evaluate if a candidate is a good fit for a role. 
                If you don't know the answer, just say that you don't know, don't try to make up an answer.
                Question: {question}
           """
)

retriever = MultiQueryRetriever.from_llm(vector_db.as_retriever(),llm, prompt=QUERY_PROMPT)

# use a ChatPromptTemplate to initiate a conversation, allowing the System to assume a Role
chat_template = """Answer the question based only on the following context: 
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(chat_template)

chain = (
    {"context":retriever, "question":RunnablePassthrough()}
    | prompt 
    | llm 
    | StrOutputParser()
)

In [6]:
# copy company_description and job_description from LinkedIn job posts
company_description = """Since its founding in 1993, NVIDIA (NASDAQ: NVDA) has been a pioneer in accelerated computing. The company’s invention of the GPU in 1999 sparked the growth of the PC gaming market, redefined computer graphics, ignited the era of modern AI and is fueling the creation of the metaverse. NVIDIA is now a full-stack computing company with data-center-scale offerings that are reshaping industry.
"""

job_description = """Professional, long-term experience as a Data Scientist (focused on AI/ML) developing/supporting business facing projects.
8+ years of experience, including 5+ years of demonstrated ability in business-focused AI and Data Science. Job scope can be adjusted to accommodate more experienced candidates.
BS/MS/PhD or equivalent experience in Computer Science, Data Science, Electrical/Computer Engineering, Physics, Mathematics, other Engineering fields. Technical Master’s or Ph.D. with finance or business background is preferred.
Data Science project management, driving projects and coordinating across multidisciplinary teams inside the organization.
Strong technical skills, with a proven history of designing, validating, deploying, and maintaining data science models using Python, SQL, & Databricks (or similar).
Excellent communication skills, with the ability to maintain good documentation and present projects to technical and business partners.
                """

In [7]:
context = f"Role: you are a recruiter for {company_description}. The job position requirements are the following:\
            {job_description}\
        "

q = "Can you evaluate the candidate in this resume document for the role provided?"

response = chain.invoke(input={'context': context, 'question': q})
print(response)

Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


Based on the provided context, I can assess the candidate's qualifications for the Data Scientist role at NVIDIA.

The candidate has 5+ years of experience in business-focused AI and Data Science, which meets the minimum requirement. They have a strong technical background in Python, PySpark, SQL, SAS, and other machine learning libraries, as well as experience with data visualization tools like NLTK, Spacy, and Word2Vec.

The candidate has also demonstrated expertise in various domains, including Federal, Telecommunications, Real Estate, SaaS, Subscription Services, and Meal Kit industries. This suggests that they have a broad understanding of different business applications and can adapt to various projects.

Furthermore, the candidate has experience working with Agile methodologies (Atlassian Jira & Confluence), managing teams (hired and managed 3 direct reports), and facilitating daily and monthly updates to SaaS products. These skills are valuable for data science project manageme

#### Evaluate the response using Ragas

In [9]:
# Set up a basic PromptTemplate as the backbones of the solution
QUERY_PROMPT = PromptTemplate(
    input_variables = ["question"],
        template="""As a job recruiter, you want to understand a candidate's resume, but not specific to a job position. 
                If you don't know the answer, just say that you don't know, don't try to make up an answer.
                Question: {question}
           """
)

retriever = MultiQueryRetriever.from_llm(vector_db.as_retriever(),llm, prompt=QUERY_PROMPT)

# use a ChatPromptTemplate to initiate a conversation, allowing the System to assume a Role
chat_template = """Answer the question based only on the following context: 
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(chat_template)

chain = (
    {"context":retriever, "question":RunnablePassthrough()}
    | prompt 
    | llm 
    | StrOutputParser()
)

In [15]:
context0 = f"Role: you are a generic recruiter reading a candidate's resume."
q0 = "Based on the dates listed for each position, how many years of experience does this candidate have?"

response0 = chain.invoke(input={'context': context0, 'question': q0})
print(response0)

To calculate the number of years of experience, we need to analyze the dates provided for each position.

1. PricewaterhouseCoopers / Associate (2010-2016): 6 years
2. Masters of Science, Statistics (2007-2009): 2 years
3. Hellofresh / Senior Data Scientist (2021-2025): 4 years

Adding up these periods: 6 + 2 + 4 = 12 years.

So, the candidate has a total of 12 years of experience.


In [16]:
q1 = "Based on this resume, what machine libraries is the candidate experienced with?"

response1 = chain.invoke(input={'context': context0, 'question': q1})
print(response1)

Based on the provided context, the candidate has experience with the following machine learning libraries:

1. TensorFlow
2. PyTorch
3. scikit-learn
4. Prophet
5. Nixtla TimeGPT
6. Statsmodels
7. Sagemaker DeepAR
8. Catboost
9. sklearn
10. NLTK
11. Spacy
12. Word2Vec

Additionally, the candidate has experience with various other machine learning models and frameworks, including:

* SBERT/HuggingFace
* OpenAl
* Langchain
* YOLO
* GCP AutoML
* Databricks MLOps


In [47]:
data_samples = {
    'question': [
         #q0, 
         q1
    ],
    'answer': [
         #response0.replace("\n", ""), 
         response1.replace("\n", "")

    ],
    'contexts': [
        [
            context0
        ],
        #[
        #    context0
        #]
    ],
    'ground_truth': [
        #'The candidate has a total of 18.5 years of experience.',
        'Tensorflow, PyTorch, scikit-learn, Prophet, Nixtla TimeGPT, Statsmodels, Sagemaker DeepAR, Catboost, sklearn, NLTK, Spacy, Word2Vec, SBERT/HuggingFace, OpenAI, Langchain, YOLO,  Deep learning, Reinforcement Learning, RAG Systems'
    ]
}

dataset = Dataset.from_dict(data_samples)

json_formatted_string = json.dumps(data_samples, indent=4)
print(json_formatted_string)

{
    "question": [
        "Based on this resume, what machine libraries is the candidate experienced with?"
    ],
    "answer": [
        "Based on the provided context, the candidate has experience with the following machine learning libraries:1. TensorFlow2. PyTorch3. scikit-learn4. Prophet5. Nixtla TimeGPT6. Statsmodels7. Sagemaker DeepAR8. Catboost9. sklearn10. NLTK11. Spacy12. Word2VecAdditionally, the candidate has experience with various other machine learning models and frameworks, including:* SBERT/HuggingFace* OpenAl* Langchain* YOLO* GCP AutoML* Databricks MLOps"
    ],
    "contexts": [
        [
            "Role: you are a generic recruiter reading a candidate's resume."
        ]
    ],
    "ground_truth": [
        "Tensorflow, PyTorch, scikit-learn, Prophet, Nixtla TimeGPT, Statsmodels, Sagemaker DeepAR, Catboost, sklearn, NLTK, Spacy, Word2Vec, SBERT/HuggingFace, OpenAI, Langchain, YOLO,  Deep learning, Reinforcement Learning, RAG Systems"
    ]
}


In [48]:
# Ensure deterministic output for evaluation
llm_eval = ChatOllama(model=local_llm, temperature=0)
evaluator_llm = LangchainLLMWrapper(llm_eval)
evaluator_embed = LangchainEmbeddingsWrapper(OllamaEmbeddings(model="nomic-embed-text"))

# Test if embeddings wrapper works
try:
    test_embedding = evaluator_embed.embed_query("test text")
    print(f"Embedding dimension: {len(test_embedding)}")
except Exception as e:
    print(f"Embeddings wrapper error: {e}")

Embedding dimension: 768


In [51]:

# Evaluate using Ragas
result = evaluate(
    dataset,
    metrics=[
        #context_precision,
        faithfulness,
        answer_relevancy,
        #context_recall
        answer_correctness,
        answer_similarity,
    ],
    llm=evaluator_llm,
    embeddings=evaluator_embed,
    run_config=RunConfig(timeout=300, 
                         max_retries=3, # Reduce retries initially
                         max_wait=300, 
                         log_tenacity=True # Enable logging for debugging
    ),
    raise_exceptions=False # Don't stop on first error
)

print(result)

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

Exception raised in Job[1]: OutputParserException(Invalid json output: What machine learning libraries and frameworks does the candidate have experience with?
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE )
Exception raised in Job[0]: TimeoutError()
Exception raised in Job[2]: TimeoutError()


{'faithfulness': nan, 'answer_relevancy': nan, 'answer_correctness': nan, 'semantic_similarity': 0.7626}
